## Silver to Gold

This notebook transforms curated Silver-layer Delta tables into analytics-ready Gold-layer datasets using a dimensional (star-schema) approach.

Key objectives of this notebook:

Build dimension tables (e.g. customer, address, product) with clean keys and descriptive attributes

Build fact tables at the correct grain (order header and order line)

Enforce consistent data types, naming, and business-friendly schemas

Remove technical or source-system-only fields not required for analytics

Write all Gold outputs as Delta tables for reliability, performance, and downstream BI consumption

### Configure secure data lake access

This cell configures Databricks to securely access Azure Data Lake Storage using OAuth so that Silver and Gold Delta tables can be read and written without using storage account keys.


In [0]:
# Configure Azure Data Lake Storage access using OAuth
# This replaces the mounting step and provides access to bronze, silver, and gold containers

spark.conf.set("fs.azure.account.auth.type.gbosstorageaccount.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.gbosstorageaccount.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.gbosstorageaccount.dfs.core.windows.net", dbutils.secrets.get(scope="my-scope", key="client-id"))
spark.conf.set("fs.azure.account.oauth2.client.secret.gbosstorageaccount.dfs.core.windows.net", dbutils.secrets.get(scope="my-scope", key="client-secret"))
spark.conf.set("fs.azure.account.oauth2.client.endpoint.gbosstorageaccount.dfs.core.windows.net", "https://login.microsoftonline.com/5d5ed5dc-15a7-49c2-99e6-d3bd5764b356/oauth2/token")

print("✓ Azure Data Lake Storage access configured")
print("  Bronze: abfss://bronze@gbosstorageaccount.dfs.core.windows.net/")
print("  Silver: abfss://silver@gbosstorageaccount.dfs.core.windows.net/")
print("  Gold:   abfss://gold@gbosstorageaccount.dfs.core.windows.net/")

✓ Azure Data Lake Storage access configured
  Bronze: abfss://bronze@gbosstorageaccount.dfs.core.windows.net/
  Silver: abfss://silver@gbosstorageaccount.dfs.core.windows.net/
  Gold:   abfss://gold@gbosstorageaccount.dfs.core.windows.net/


### Validate Silver layer availability

This cell lists the contents of the Silver container to confirm that all expected curated Delta tables are available before Gold transformations begin.


In [0]:
dbutils.fs.ls('abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/')

[FileInfo(path='abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/Address/', name='Address/', size=0, modificationTime=1765895701000),
 FileInfo(path='abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/Customer/', name='Customer/', size=0, modificationTime=1765895719000),
 FileInfo(path='abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/CustomerAddress/', name='CustomerAddress/', size=0, modificationTime=1765895725000),
 FileInfo(path='abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/Product/', name='Product/', size=0, modificationTime=1765895731000),
 FileInfo(path='abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/ProductCategory/', name='ProductCategory/', size=0, modificationTime=1765895737000),
 FileInfo(path='abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/ProductDescription/', name='ProductDescription/', size=0, modificationTime=1765895742000),
 FileInfo(path='abfss://silver@gbosstorageaccount.dfs.co

In [0]:
files = dbutils.fs.ls(
    "abfss://silver@gbosstorageaccount.dfs.core.windows.net/"
)
if files:
    display(files)
else:
    print("No files found in the specified path.")

path,name,size,modificationTime
abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/,SalesLT/,0,1765895701000


### Load Silver Customer table

This cell reads the curated Silver Customer Delta table into a Spark DataFrame for downstream dimensional modelling.


In [0]:
df_customer = spark.read.format('delta').load('abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/Customer/')
display(df_customer)

customer_id,name_style,title,first_name,middle_name,last_name,suffix,company_name,sales_person,email_address,phone,password_hash,password_salt,modified_date
1,false,Mr.,Orlando,N.,Gee,null,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,2005-08-01
2,false,Mr.,Keith,null,Harris,null,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,2006-08-01
3,false,Ms.,Donna,F.,Carreras,null,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,2005-09-01
4,false,Ms.,Janet,M.,Gates,null,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,2006-07-01
5,false,Mr.,Lucy,null,Harrington,null,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,2006-09-01
6,false,Ms.,Rosmarie,J.,Carroll,null,Aerobic Exercise Company,adventure-works\linda3,rosmarie0@adventure-works.com,244-555-0112,OKT0scizCdIzymHHOtyJKQiC/fCILSooSZ8dQ2Y34VM=,ihWf50M=,2007-09-01
7,false,Mr.,Dominic,P.,Gash,null,Associated Bikes,adventure-works\shu0,dominic0@adventure-works.com,192-555-0173,ZccoP/jZGQm+Xpzc7RKwDhS11YFNybwcPVRYTSNcnSg=,sPoUBSQ=,2006-07-01
10,false,Ms.,Kathleen,M.,Garza,null,Rural Cycle Emporium,adventure-works\josé1,kathleen0@adventure-works.com,150-555-0127,Qa3aMCxNbVLGrc0b99KsbQqiVgwYDfHcsK9GZSUxcTM=,Ls05W3g=,2006-09-01
11,false,Ms.,Katherine,null,Harding,null,Sharp Bikes,adventure-works\josé1,katherine0@adventure-works.com,926-555-0159,uRlorVzDGNJIX9I+ehTlRK+liT4UKRgWhApJgUMC2d4=,jpHKbqE=,2005-08-01
12,false,Mr.,Johnny,A.,Caprio,Jr.,Bikes and Motorbikes,adventure-works\garrett1,johnny0@adventure-works.com,112-555-0191,jtF9jBoFYeJTaET7x+eJDkd7BzMz15Wo9odbGPBaIak=,wVLnvHo=,2006-08-01


### Build Customer dimension (dm_customer)

This cell creates the Customer dimension by selecting business-friendly customer attributes, generating a clean customer key, and preparing the data for analytics use.


In [0]:
from pyspark.sql import functions as F

# --- Paths ---
silver_customer_path = "abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/Customer/"
gold_dm_customer_path = "abfss://gold@gbosstorageaccount.dfs.core.windows.net/SalesLT/dm_customer/"

# --- Read Silver ---
df_customer = spark.read.format("delta").load(silver_customer_path)

# --- Build dm_customer (dimension) ---
dm_customer = (
    df_customer
    .select(
        F.col("customer_id").cast("int").alias("customer_key"),
        F.col("title"),
        F.col("first_name"),
        F.col("middle_name"),
        F.col("last_name"),
        F.col("suffix"),
        F.col("company_name"),
        F.col("sales_person"),
        F.col("email_address"),
        F.col("phone"),
        F.col("modified_date")
    )
    # nice-to-have attributes for a dim table
    .withColumn(
        "full_name",
        F.trim(
            F.concat_ws(
                " ",
                F.col("title"),
                F.col("first_name"),
                F.col("middle_name"),
                F.col("last_name"),
                F.col("suffix")
            )
        )
    )
    .withColumn("email_address", F.lower(F.col("email_address")))
    .dropDuplicates(["customer_key"])  # keep 1 row per customer
)

In [0]:
display(dm_customer)

customer_key,title,first_name,middle_name,last_name,suffix,company_name,sales_person,email_address,phone,modified_date,full_name
1,Mr.,Orlando,N.,Gee,null,A Bike Store,adventure-works\pamela0,orlando0@adventure-works.com,245-555-0173,2005-08-01,Mr. Orlando N. Gee
2,Mr.,Keith,null,Harris,null,Progressive Sports,adventure-works\david8,keith0@adventure-works.com,170-555-0127,2006-08-01,Mr. Keith Harris
3,Ms.,Donna,F.,Carreras,null,Advanced Bike Components,adventure-works\jillian0,donna0@adventure-works.com,279-555-0130,2005-09-01,Ms. Donna F. Carreras
4,Ms.,Janet,M.,Gates,null,Modular Cycle Systems,adventure-works\jillian0,janet1@adventure-works.com,710-555-0173,2006-07-01,Ms. Janet M. Gates
5,Mr.,Lucy,null,Harrington,null,Metropolitan Sports Supply,adventure-works\shu0,lucy0@adventure-works.com,828-555-0186,2006-09-01,Mr. Lucy Harrington
6,Ms.,Rosmarie,J.,Carroll,null,Aerobic Exercise Company,adventure-works\linda3,rosmarie0@adventure-works.com,244-555-0112,2007-09-01,Ms. Rosmarie J. Carroll
7,Mr.,Dominic,P.,Gash,null,Associated Bikes,adventure-works\shu0,dominic0@adventure-works.com,192-555-0173,2006-07-01,Mr. Dominic P. Gash
10,Ms.,Kathleen,M.,Garza,null,Rural Cycle Emporium,adventure-works\josé1,kathleen0@adventure-works.com,150-555-0127,2006-09-01,Ms. Kathleen M. Garza
11,Ms.,Katherine,null,Harding,null,Sharp Bikes,adventure-works\josé1,katherine0@adventure-works.com,926-555-0159,2005-08-01,Ms. Katherine Harding
12,Mr.,Johnny,A.,Caprio,Jr.,Bikes and Motorbikes,adventure-works\garrett1,johnny0@adventure-works.com,112-555-0191,2006-08-01,Mr. Johnny A. Caprio Jr.


### Load Silver Address table

This cell loads the Silver Address Delta table to prepare address attributes for dimensional modelling.


In [0]:
df_address = spark.read.format('delta')
df_address = spark.read.format('delta').load('abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/Address/')
display(df_address)

address_id,address_line_1,address_line_2,city,state_province,country_region,postal_code,modified_date
9,8713 Yosemite Ct.,null,Bothell,Washington,United States,98011,2006-07-01
11,1318 Lasalle Street,null,Bothell,Washington,United States,98011,2007-04-01
25,9178 Jumping St.,null,Dallas,Texas,United States,75201,2006-09-01
28,9228 Via Del Sol,null,Phoenix,Arizona,United States,85004,2005-09-01
32,26910 Indela Road,null,Montreal,Quebec,Canada,H1Y 2H5,2006-08-01
185,2681 Eagle Peak,null,Bellevue,Washington,United States,98004,2006-09-01
297,7943 Walnut Ave,null,Renton,Washington,United States,98055,2006-08-01
445,6388 Lake City Way,null,Burnaby,British Columbia,Canada,V5A 3A6,2006-09-01
446,52560 Free Street,null,Toronto,Ontario,Canada,M4B 1V7,2005-08-01
447,22580 Free Street,null,Toronto,Ontario,Canada,M4B 1V7,2006-08-01


### Build Address dimension (dm_address)

This cell constructs the Address dimension by selecting location-related attributes and ensuring one row per address.


In [0]:
from pyspark.sql import functions as F

# --- Paths ---
silver_address_path = "abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/Address/"
gold_dm_address_path = "abfss://gold@gbosstorageaccount.dfs.core.windows.net/SalesLT/dm_address/"

# --- Read Silver ---
df_address = spark.read.format("delta").load(silver_address_path)

# --- Build dm_address ---
dm_address = (
    df_address
    .select(
        F.col("address_id").cast("int").alias("address_key"),
        F.col("address_line_1"),
        F.col("address_line_2"),
        F.col("city"),
        F.col("state_province"),
        F.col("country_region"),
        F.col("postal_code"),
        F.col("modified_date")
    )
    .dropDuplicates(["address_key"])  # one row per address
)

# --- Optional validation ---
display(dm_address)
dm_address.printSchema()


address_key,address_line_1,address_line_2,city,state_province,country_region,postal_code,modified_date
9,8713 Yosemite Ct.,null,Bothell,Washington,United States,98011,2006-07-01
11,1318 Lasalle Street,null,Bothell,Washington,United States,98011,2007-04-01
25,9178 Jumping St.,null,Dallas,Texas,United States,75201,2006-09-01
28,9228 Via Del Sol,null,Phoenix,Arizona,United States,85004,2005-09-01
32,26910 Indela Road,null,Montreal,Quebec,Canada,H1Y 2H5,2006-08-01
185,2681 Eagle Peak,null,Bellevue,Washington,United States,98004,2006-09-01
297,7943 Walnut Ave,null,Renton,Washington,United States,98055,2006-08-01
445,6388 Lake City Way,null,Burnaby,British Columbia,Canada,V5A 3A6,2006-09-01
446,52560 Free Street,null,Toronto,Ontario,Canada,M4B 1V7,2005-08-01
447,22580 Free Street,null,Toronto,Ontario,Canada,M4B 1V7,2006-08-01


root
 |-- address_key: integer (nullable = true)
 |-- address_line_1: string (nullable = true)
 |-- address_line_2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state_province: string (nullable = true)
 |-- country_region: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- modified_date: date (nullable = true)



### Load Silver CustomerAddress table

This cell reads the Silver CustomerAddress Delta table, which represents the relationship between customers and addresses.


In [0]:
df_customer_address = spark.read.format('delta')
df_customer_address = spark.read.format('delta').load('abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/CustomerAddress/')
display(df_customer_address)

customer_id,address_id,address_type,modified_date
29485,1086,Main Office,2007-09-01
29486,621,Main Office,2005-09-01
29489,1069,Main Office,2005-07-01
29490,887,Main Office,2006-09-01
29492,618,Main Office,2006-12-01
29494,537,Main Office,2005-09-01
29496,1072,Main Office,2007-09-01
29497,889,Main Office,2005-07-01
29499,527,Main Office,2006-09-01
29502,893,Main Office,2007-07-01


### Build Customer–Address bridge dimension

This cell creates a bridge table that models the many-to-many relationship between customers and addresses, preserving address types.


In [0]:
from pyspark.sql import functions as F

# --- Paths ---
silver_customer_address_path = "abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/CustomerAddress/"
gold_dm_customer_address_path = "abfss://gold@gbosstorageaccount.dfs.core.windows.net/SalesLT/dm_customer_address/"

# --- Read Silver ---
df_customer_address = spark.read.format("delta").load(silver_customer_address_path)

# --- Build dm_customer_address (bridge / relationship table) ---
dm_customer_address = (
    df_customer_address
    .select(
        F.col("customer_id").cast("int").alias("customer_key"),
        F.col("address_id").cast("int").alias("address_key"),
        F.trim(F.col("address_type")).alias("address_type"),
        F.col("modified_date")
    )
    # keep unique relationship rows
    .dropDuplicates(["customer_key", "address_key", "address_type"])
)

# --- Optional validation ---
display(dm_customer_address)
dm_customer_address.printSchema()

customer_key,address_key,address_type,modified_date
29550,853,Main Office,2007-09-01
29753,897,Main Office,2006-08-01
30033,1090,Main Office,2007-09-01
30010,535,Main Office,2005-11-01
29819,633,Main Office,2006-08-01
29953,846,Main Office,2006-08-01
29532,881,Main Office,2006-10-01
29600,797,Main Office,2006-09-01
29789,501,Main Office,2005-08-01
29981,904,Main Office,2006-07-01


root
 |-- customer_key: integer (nullable = true)
 |-- address_key: integer (nullable = true)
 |-- address_type: string (nullable = true)
 |-- modified_date: date (nullable = true)



### Load Silver Product table

This cell loads the Silver Product Delta table containing product attributes and pricing details.


In [0]:
df_product = spark.read.format('delta')
df_product = spark.read.format('delta').load('abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/Product/')
display(df_product)

product_id,name,product_number,color,standard_cost,list_price,size,weight,product_category_id,product_model_id,sell_start_date,sell_end_date,discontinued_date,thumb_nail_photo,thumbnail_photo_file_name,modified_date
680,"HL Road Frame - Black, 58",FR-R92B-58,Black,1059.3100,1431.5000,58,1016.04,18,6,2002-06-01,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,2008-03-11
706,"HL Road Frame - Red, 58",FR-R92R-58,Red,1059.3100,1431.5000,58,1016.04,18,6,2002-06-01,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,2008-03-11
707,"Sport-100 Helmet, Red",HL-U509-R,Red,13.0863,34.9900,null,null,35,33,2005-07-01,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,2008-03-11
708,"Sport-100 Helmet, Black",HL-U509,Black,13.0863,34.9900,null,null,35,33,2005-07-01,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,2008-03-11
709,"Mountain Bike Socks, M",SO-B909-M,White,3.3963,9.5000,M,null,27,18,2005-07-01,2006-06-30,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,2008-03-11
710,"Mountain Bike Socks, L",SO-B909-L,White,3.3963,9.5000,L,null,27,18,2005-07-01,2006-06-30,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,2008-03-11
711,"Sport-100 Helmet, Blue",HL-U509-B,Blue,13.0863,34.9900,null,null,35,33,2005-07-01,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,2008-03-11
712,AWC Logo Cap,CA-1098,Multi,6.9223,8.9900,null,null,23,2,2005-07-01,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA= (truncated),no_image_available_small.gif,2008-03-11
713,"Long-Sleeve Logo Jersey, S",LJ-0192-S,Multi,38.4923,49.9900,S,null,25,11,2005-07-01,null,null,R0lGODlhUAAyAPcAABQlZ/z8/J+fnyxLZ87OzuTk5Dtliqqqqiw0WMrKyvDw8EdXpRAYLsbGxubm5urq6qSkpO7u7uLi4mlmZa+vrxAhNiJai4iVqtLQ5rGxsY2v0zMzNfr6+i5zsiVLcmNwmLK8yHZ1esLCwhhHnfb29lB1rDI= (truncated),awc_jersey_male_small.gif,2008-03-11
714,"Long-Sleeve Logo Jersey, M",LJ-0192-M,Multi,38.4923,49.9900,M,null,25,11,2005-07-01,null,null,R0lGODlhUAAyAPcAABQlZ/z8/J+fnyxLZ87OzuTk5Dtliqqqqiw0WMrKyvDw8EdXpRAYLsbGxubm5urq6qSkpO7u7uLi4mlmZa+vrxAhNiJai4iVqtLQ5rGxsY2v0zMzNfr6+i5zsiVLcmNwmLK8yHZ1esLCwhhHnfb29lB1rDI= (truncated),awc_jersey_male_small.gif,2008-03-11


### Build Product dimension (dm_product)

This cell builds the Product dimension by selecting descriptive product attributes and deriving useful analytical measures such as margin.


In [0]:
from pyspark.sql import functions as F

# --- Paths ---
silver_product_path = "abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/Product/"
gold_dm_product_path = "abfss://gold@gbosstorageaccount.dfs.core.windows.net/SalesLT/dm_product/"

# --- Read Silver ---
df_product = spark.read.format("delta").load(silver_product_path)

# --- Build dm_product (dimension) ---
dm_product = (
    df_product
    .select(
        F.col("product_id").cast("int").alias("product_key"),
        F.col("name").alias("product_name"),
        F.col("product_number"),
        F.col("color"),
        F.col("size"),
        F.col("weight").cast("double").alias("weight"),
        F.col("standard_cost").cast("double").alias("standard_cost"),
        F.col("list_price").cast("double").alias("list_price"),
        F.col("product_category_id").cast("int").alias("product_category_key"),
        F.col("product_model_id").cast("int").alias("product_model_key"),
        F.col("sell_start_date"),
        F.col("sell_end_date"),
        F.col("modified_date")
    )
    # useful derived fields
    .withColumn("margin", (F.col("list_price") - F.col("standard_cost")).cast("double"))
    .dropDuplicates(["product_key"])
)

# --- Validate ---
display(dm_product)
dm_product.printSchema()


product_key,product_name,product_number,color,size,weight,standard_cost,list_price,product_category_key,product_model_key,sell_start_date,sell_end_date,modified_date,margin
680,"HL Road Frame - Black, 58",FR-R92B-58,Black,58,1016.04,1059.31,1431.5,18,6,2002-06-01,null,2008-03-11,372.19000000000005
706,"HL Road Frame - Red, 58",FR-R92R-58,Red,58,1016.04,1059.31,1431.5,18,6,2002-06-01,null,2008-03-11,372.19000000000005
707,"Sport-100 Helmet, Red",HL-U509-R,Red,null,null,13.0863,34.99,35,33,2005-07-01,null,2008-03-11,21.9037
708,"Sport-100 Helmet, Black",HL-U509,Black,null,null,13.0863,34.99,35,33,2005-07-01,null,2008-03-11,21.9037
709,"Mountain Bike Socks, M",SO-B909-M,White,M,null,3.3963,9.5,27,18,2005-07-01,2006-06-30,2008-03-11,6.1037
710,"Mountain Bike Socks, L",SO-B909-L,White,L,null,3.3963,9.5,27,18,2005-07-01,2006-06-30,2008-03-11,6.1037
711,"Sport-100 Helmet, Blue",HL-U509-B,Blue,null,null,13.0863,34.99,35,33,2005-07-01,null,2008-03-11,21.9037
712,AWC Logo Cap,CA-1098,Multi,null,null,6.9223,8.99,23,2,2005-07-01,null,2008-03-11,2.0677000000000003
713,"Long-Sleeve Logo Jersey, S",LJ-0192-S,Multi,S,null,38.4923,49.99,25,11,2005-07-01,null,2008-03-11,11.497700000000002
714,"Long-Sleeve Logo Jersey, M",LJ-0192-M,Multi,M,null,38.4923,49.99,25,11,2005-07-01,null,2008-03-11,11.497700000000002


root
 |-- product_key: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_number: string (nullable = true)
 |-- color: string (nullable = true)
 |-- size: string (nullable = true)
 |-- weight: double (nullable = true)
 |-- standard_cost: double (nullable = true)
 |-- list_price: double (nullable = true)
 |-- product_category_key: integer (nullable = true)
 |-- product_model_key: integer (nullable = true)
 |-- sell_start_date: date (nullable = true)
 |-- sell_end_date: date (nullable = true)
 |-- modified_date: date (nullable = true)
 |-- margin: double (nullable = true)



### Load Silver ProductCategory table

This cell reads the Silver ProductCategory Delta table, which defines the product category hierarchy.


In [0]:
df_product_category = spark.read.format('delta')
df_product_category = spark.read.format('delta').load('abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/ProductCategory/')
display(df_product_category)

product_category_id,parent_product_category_id,name,modified_date
1,null,Bikes,2002-06-01
2,null,Components,2002-06-01
3,null,Clothing,2002-06-01
4,null,Accessories,2002-06-01
5,1,Mountain Bikes,2002-06-01
6,1,Road Bikes,2002-06-01
7,1,Touring Bikes,2002-06-01
8,2,Handlebars,2002-06-01
9,2,Bottom Brackets,2002-06-01
10,2,Brakes,2002-06-01


### Build Product Category dimension (dm_product_category)

This cell creates the Product Category dimension, including parent–child relationships for hierarchical analysis.


In [0]:
from pyspark.sql import functions as F

# --- Paths ---
silver_product_category_path = (
    "abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/ProductCategory/"
)

gold_dm_product_category_path = (
    "abfss://gold@gbosstorageaccount.dfs.core.windows.net/SalesLT/dm_product_category/"
)

# --- Read Silver ---
df_product_category = (
    spark.read
    .format("delta")
    .load(silver_product_category_path)
)

# --- Build dm_product_category ---
dm_product_category = (
    df_product_category
    .select(
        F.col("product_category_id").cast("int").alias("product_category_key"),
        F.col("name").alias("product_category_name"),
        F.col("parent_product_category_id").cast("int").alias("parent_product_category_key"),
        F.col("modified_date")
    )
    .dropDuplicates(["product_category_key"])   # one row per category
)

# --- Optional validation ---
display(dm_product_category)
dm_product_category.printSchema()


product_category_key,product_category_name,parent_product_category_key,modified_date
1,Bikes,null,2002-06-01
2,Components,null,2002-06-01
3,Clothing,null,2002-06-01
4,Accessories,null,2002-06-01
5,Mountain Bikes,1,2002-06-01
6,Road Bikes,1,2002-06-01
7,Touring Bikes,1,2002-06-01
8,Handlebars,2,2002-06-01
9,Bottom Brackets,2,2002-06-01
10,Brakes,2,2002-06-01


root
 |-- product_category_key: integer (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- parent_product_category_key: integer (nullable = true)
 |-- modified_date: date (nullable = true)



### Load Silver Product Description table

This cell loads the Silver ProductDescription Delta table containing textual product descriptions.


In [0]:
df_product_description = spark.read.format('delta')
df_product_description = spark.read.format('delta').load('abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/ProductDescription/')
display(df_product_description)

product_description_id,description,modified_date
3,Chromoly steel.,2007-06-01
4,Aluminum alloy cups; large diameter spindle.,2007-06-01
5,Aluminum alloy cups and a hollow axle.,2007-06-01
8,"Suitable for any type of riding, on or off-road. Fits any budget. Smooth-shifting with a comfortable ride.",2007-06-01
64,"This bike delivers a high-level of performance on a budget. It is responsive and maneuverable, and offers peace-of-mind when you decide to go off-road.",2007-06-01
88,For true trail addicts. An extremely durable bike that will go anywhere and keep you in control on challenging terrain - without breaking your budget.,2007-06-01
128,Serious back-country riding. Perfect for all levels of competition. Uses the same HL Frame as the Mountain-100.,2008-03-11
168,"Top-of-the-line competition mountain bike. Performance-enhancing options include the innovative HL Frame, super-smooth front suspension, and traction for all terrain.",2007-06-01
170,Suitable for any type of off-road trip. Fits any budget.,2007-06-01
209,Entry level adult bike; offers a comfortable ride cross-country or down the block. Quick-release hubs and rims.,2007-06-01


### Build Product Description dimension (dm_product_description)

This cell constructs a Product Description dimension to support descriptive and multilingual product reporting.


In [0]:
silver_product_description_path = (
    "abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/ProductDescription/"
)

# --- Read Silver ---
df_product_description = (
    spark.read
    .format("delta")
    .load(silver_product_description_path)
)

# --- Build dm_product_description ---
dm_product_description = (
    df_product_description
    .select(
        F.col("product_description_id").cast("int").alias("product_description_key"),
        F.col("description").alias("product_description"),
        F.col("modified_date")
    )
    .dropDuplicates(["product_description_key"])   # one row per description
)

# --- Optional validation ---
display(dm_product_description)
dm_product_description.printSchema()


product_description_key,product_description,modified_date
3,Chromoly steel.,2007-06-01
4,Aluminum alloy cups; large diameter spindle.,2007-06-01
5,Aluminum alloy cups and a hollow axle.,2007-06-01
8,"Suitable for any type of riding, on or off-road. Fits any budget. Smooth-shifting with a comfortable ride.",2007-06-01
64,"This bike delivers a high-level of performance on a budget. It is responsive and maneuverable, and offers peace-of-mind when you decide to go off-road.",2007-06-01
88,For true trail addicts. An extremely durable bike that will go anywhere and keep you in control on challenging terrain - without breaking your budget.,2007-06-01
128,Serious back-country riding. Perfect for all levels of competition. Uses the same HL Frame as the Mountain-100.,2008-03-11
168,"Top-of-the-line competition mountain bike. Performance-enhancing options include the innovative HL Frame, super-smooth front suspension, and traction for all terrain.",2007-06-01
170,Suitable for any type of off-road trip. Fits any budget.,2007-06-01
209,Entry level adult bike; offers a comfortable ride cross-country or down the block. Quick-release hubs and rims.,2007-06-01


root
 |-- product_description_key: integer (nullable = true)
 |-- product_description: string (nullable = true)
 |-- modified_date: date (nullable = true)



### Load Silver Product Model table

This cell reads the Silver ProductModel Delta table containing product model metadata.


In [0]:
df_product_model = spark.read.format('delta')
df_product_model = spark.read.format('delta').load('abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/ProductModel/')
display(df_product_model)

product_model_id,name,catalog_description,modified_date
1,Classic Vest,null,2007-06-01
2,Cycling Cap,null,2005-06-01
3,Full-Finger Gloves,null,2006-06-01
4,Half-Finger Gloves,null,2006-06-01
5,HL Mountain Frame,null,2005-06-01
6,HL Road Frame,null,2002-05-02
7,HL Touring Frame,null,2009-05-16
8,LL Mountain Frame,null,2006-11-20
9,LL Road Frame,null,2005-06-01
10,LL Touring Frame,null,2009-05-16


### Build Product Model dimension (dm_product_model)

This cell builds the Product Model dimension by selecting model names, descriptions, and metadata for analysis.


In [0]:
silver_product_model_path = (
    "abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/ProductModel/"
)

# --- Read Silver ---
df_product_model = (
    spark.read
    .format("delta")
    .load(silver_product_model_path)
)

# --- Build dm_product_model ---
dm_product_model = (
    df_product_model
    .select(
        F.col("product_model_id").cast("int").alias("product_model_key"),
        F.col("name").alias("product_model_name"),
        F.col("catalog_description"),
        F.col("modified_date")
    )
    .dropDuplicates(["product_model_key"])  # one row per product model
)

# --- Optional validation ---
display(dm_product_model)
dm_product_model.printSchema()


product_model_key,product_model_name,catalog_description,modified_date
1,Classic Vest,null,2007-06-01
2,Cycling Cap,null,2005-06-01
3,Full-Finger Gloves,null,2006-06-01
4,Half-Finger Gloves,null,2006-06-01
5,HL Mountain Frame,null,2005-06-01
6,HL Road Frame,null,2002-05-02
7,HL Touring Frame,null,2009-05-16
8,LL Mountain Frame,null,2006-11-20
9,LL Road Frame,null,2005-06-01
10,LL Touring Frame,null,2009-05-16


root
 |-- product_model_key: integer (nullable = true)
 |-- product_model_name: string (nullable = true)
 |-- catalog_description: string (nullable = true)
 |-- modified_date: date (nullable = true)



### Load Silver Product Model table

This cell reads the Silver ProductModel Delta table containing product model metadata.


In [0]:
df_product_model_desc = spark.read.format('delta')
df_product_model_desc = spark.read.format('delta').load('abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/ProductModelProductDescription/')
display(df_product_model_desc)

product_model_id,product_description_id,culture,modified_date
1,1199,en,2007-06-01
1,1467,ar,2007-06-01
1,1589,fr,2007-06-01
1,1712,th,2007-06-01
1,1838,he,2007-06-01
1,1965,zh-cht,2007-06-01
2,1210,en,2007-06-01
2,1476,ar,2007-06-01
2,1598,fr,2007-06-01
2,1721,th,2007-06-01


### Build Product Model dimension (dm_product_model)

This cell builds the Product Model dimension by selecting model names, descriptions, and metadata for analysis.


In [0]:
from pyspark.sql import functions as F

# --- Read Silver ---
silver_product_model_product_description_path = (
    "abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/ProductModelProductDescription/"
)

df_product_model_product_description = (
    spark.read
    .format("delta")
    .load(silver_product_model_product_description_path)
)

# --- Build bridge table ---
dm_product_model_product_description = (
    df_product_model_product_description
    .select(
        F.col("product_model_id").cast("int").alias("product_model_key"),
        F.col("product_description_id").cast("int").alias("product_description_key"),
        F.col("culture"),
        F.col("modified_date")
    )
    # ensure uniqueness of relationships
    .dropDuplicates(
        ["product_model_key", "product_description_key", "culture"]
    )
)

# --- Optional validation ---
display(dm_product_model_product_description)
dm_product_model_product_description.printSchema()


product_model_key,product_description_key,culture,modified_date
56,1901,zh-cht,2007-06-01
58,1530,fr,2007-06-01
70,1794,he,2007-06-01
83,889,en,2007-06-01
88,1564,fr,2007-06-01
126,2000,ar,2007-06-01
126,2001,fr,2007-06-01
6,1573,fr,2007-06-01
26,1500,fr,2007-06-01
110,1471,ar,2007-06-01


root
 |-- product_model_key: integer (nullable = true)
 |-- product_description_key: integer (nullable = true)
 |-- culture: string (nullable = true)
 |-- modified_date: date (nullable = true)



### Load Silver Sales Order Header table

This cell reads the Silver SalesOrderHeader Delta table, which represents sales orders at header (order-level) grain.


In [0]:
df_sales_order_header = spark.read.format('delta')
df_sales_order_header = spark.read.format('delta').load('abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/SalesOrderHeader/')
display(df_sales_order_header)

sales_order_id,revision_number,order_date,due_date,ship_date,status,online_order_flag,sales_order_number,purchase_order_number,account_number,customer_id,ship_to_address_id,bill_to_address_id,ship_method,credit_card_approval_code,sub_total,tax_amt,freight,total_due,comment,modified_date
71774,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71774,PO348186287,10-4020-000609,29847,1092,1092,CARGO TRANSPORT 5,null,880.3484,70.4279,22.0087,972.7850,null,2008-06-08
71776,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71776,PO19952192051,10-4020-000106,30072,640,640,CARGO TRANSPORT 5,null,78.8100,6.3048,1.9703,87.0851,null,2008-06-08
71780,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71780,PO19604173239,10-4020-000340,30113,653,653,CARGO TRANSPORT 5,null,38418.6895,3073.4952,960.4672,42452.6519,null,2008-06-08
71782,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71782,PO19372114749,10-4020-000582,29485,1086,1086,CARGO TRANSPORT 5,null,39785.3304,3182.8264,994.6333,43962.7901,null,2008-06-08
71783,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71783,PO19343113609,10-4020-000024,29957,992,992,CARGO TRANSPORT 5,null,83858.4261,6708.6741,2096.4607,92663.5609,null,2008-06-08
71784,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71784,PO19285135919,10-4020-000448,29736,659,659,CARGO TRANSPORT 5,null,108561.8317,8684.9465,2714.0458,119960.8240,null,2008-06-08
71796,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71796,PO17052159664,10-4020-000420,29660,1058,1058,CARGO TRANSPORT 5,null,57634.6342,4610.7707,1440.8659,63686.2708,null,2008-06-08
71797,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71797,PO16501134889,10-4020-000142,29796,642,642,CARGO TRANSPORT 5,null,78029.6898,6242.3752,1950.7422,86222.8072,null,2008-06-08
71815,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71815,PO13021155785,10-4020-000276,30089,1034,1034,CARGO TRANSPORT 5,null,1141.5782,91.3263,28.5395,1261.4440,null,2008-06-08
71816,2,2008-06-01,2008-06-13,2008-06-08,5,false,SO71816,PO12992180445,10-4020-000295,30027,1038,1038,CARGO TRANSPORT 5,null,3398.1659,271.8533,84.9541,3754.9733,null,2008-06-08


### Build Sales Order Header fact table

This cell creates a fact table at the sales order header level, capturing order dates, customer keys, and monetary measures.


In [0]:
from pyspark.sql import functions as F

# --- Paths ---
silver_sales_order_header_path = "abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/SalesOrderHeader/"
gold_fct_sales_order_header_path = "abfss://gold@gbosstorageaccount.dfs.core.windows.net/SalesLT/fct_sales_order_header/"

# --- Read Silver ---
df_sales_order_header = spark.read.format("delta").load(silver_sales_order_header_path)

# --- Build fact table (header-level) ---
fct_sales_order_header = (
    df_sales_order_header
    .select(
        F.col("sales_order_id").cast("int").alias("sales_order_key"),
        F.col("customer_id").cast("int").alias("customer_key"),
        F.col("ship_to_address_id").cast("int").alias("ship_to_address_key"),
        F.col("bill_to_address_id").cast("int").alias("bill_to_address_key"),

        F.col("order_date"),
        F.col("due_date"),
        F.col("ship_date"),

        F.col("status").cast("int").alias("status"),
        F.col("online_order_flag").cast("boolean").alias("online_order_flag"),

        F.col("sales_order_number"),
        F.col("purchase_order_number"),
        F.col("account_number"),

        # amounts (cast defensively)
        F.col("sub_total").cast("double").alias("sub_total"),
        F.col("tax_amt").cast("double").alias("tax_amt"),
        F.col("freight").cast("double").alias("freight"),
        F.col("total_due").cast("double").alias("total_due"),

        F.col("modified_date")
    )
    .dropDuplicates(["sales_order_key"])
)

# --- Optional validation ---
display(fct_sales_order_header)
fct_sales_order_header.printSchema()


sales_order_key,customer_key,ship_to_address_key,bill_to_address_key,order_date,due_date,ship_date,status,online_order_flag,sales_order_number,purchase_order_number,account_number,sub_total,tax_amt,freight,total_due,modified_date
71774,29847,1092,1092,2008-06-01,2008-06-13,2008-06-08,5,false,SO71774,PO348186287,10-4020-000609,880.3484,70.4279,22.0087,972.785,2008-06-08
71776,30072,640,640,2008-06-01,2008-06-13,2008-06-08,5,false,SO71776,PO19952192051,10-4020-000106,78.81,6.3048,1.9703,87.0851,2008-06-08
71780,30113,653,653,2008-06-01,2008-06-13,2008-06-08,5,false,SO71780,PO19604173239,10-4020-000340,38418.6895,3073.4952,960.4672,42452.6519,2008-06-08
71782,29485,1086,1086,2008-06-01,2008-06-13,2008-06-08,5,false,SO71782,PO19372114749,10-4020-000582,39785.3304,3182.8264,994.6333,43962.7901,2008-06-08
71783,29957,992,992,2008-06-01,2008-06-13,2008-06-08,5,false,SO71783,PO19343113609,10-4020-000024,83858.4261,6708.6741,2096.4607,92663.5609,2008-06-08
71784,29736,659,659,2008-06-01,2008-06-13,2008-06-08,5,false,SO71784,PO19285135919,10-4020-000448,108561.8317,8684.9465,2714.0458,119960.824,2008-06-08
71796,29660,1058,1058,2008-06-01,2008-06-13,2008-06-08,5,false,SO71796,PO17052159664,10-4020-000420,57634.6342,4610.7707,1440.8659,63686.2708,2008-06-08
71797,29796,642,642,2008-06-01,2008-06-13,2008-06-08,5,false,SO71797,PO16501134889,10-4020-000142,78029.6898,6242.3752,1950.7422,86222.8072,2008-06-08
71815,30089,1034,1034,2008-06-01,2008-06-13,2008-06-08,5,false,SO71815,PO13021155785,10-4020-000276,1141.5782,91.3263,28.5395,1261.444,2008-06-08
71816,30027,1038,1038,2008-06-01,2008-06-13,2008-06-08,5,false,SO71816,PO12992180445,10-4020-000295,3398.1659,271.8533,84.9541,3754.9733,2008-06-08


root
 |-- sales_order_key: integer (nullable = true)
 |-- customer_key: integer (nullable = true)
 |-- ship_to_address_key: integer (nullable = true)
 |-- bill_to_address_key: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- due_date: date (nullable = true)
 |-- ship_date: date (nullable = true)
 |-- status: integer (nullable = true)
 |-- online_order_flag: boolean (nullable = true)
 |-- sales_order_number: string (nullable = true)
 |-- purchase_order_number: string (nullable = true)
 |-- account_number: string (nullable = true)
 |-- sub_total: double (nullable = true)
 |-- tax_amt: double (nullable = true)
 |-- freight: double (nullable = true)
 |-- total_due: double (nullable = true)
 |-- modified_date: date (nullable = true)



### Load Silver Sales Order Detail table

This cell loads the Silver SalesOrderDetail Delta table containing line-level sales transaction data.


In [0]:
df_sales_order_detail = spark.read.format('delta')
df_sales_order_detail = spark.read.format('delta').load('abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/SalesOrderDetail/')
display(df_sales_order_detail)

sales_order_id,sales_order_detail_id,order_qty,product_id,unit_price,unit_price_discount,line_total,modified_date
71774,110562,1,836,356.8980,0.0000,356.898000,2008-06-01
71774,110563,1,822,356.8980,0.0000,356.898000,2008-06-01
71776,110567,1,907,63.9000,0.0000,63.900000,2008-06-01
71780,110616,4,905,218.4540,0.0000,873.816000,2008-06-01
71780,110617,2,983,461.6940,0.0000,923.388000,2008-06-01
71780,110618,6,988,112.9980,0.4000,406.792800,2008-06-01
71780,110619,2,748,818.7000,0.0000,1637.400000,2008-06-01
71780,110620,1,990,323.9940,0.0000,323.994000,2008-06-01
71780,110621,1,926,149.8740,0.0000,149.874000,2008-06-01
71780,110622,1,743,809.7600,0.0000,809.760000,2008-06-01


### Build Sales Order Detail fact table

This cell builds the line-level sales fact table, calculating quantities, prices, discounts, and derived financial metrics.


In [0]:
from pyspark.sql import functions as F

# --- Paths ---
silver_sales_order_detail_path = "abfss://silver@gbosstorageaccount.dfs.core.windows.net/SalesLT/SalesOrderDetail/"

# --- Read Silver ---
df_sales_order_detail = spark.read.format("delta").load(silver_sales_order_detail_path)

# --- Build fact table (detail / line-level) ---
fct_sales_order_detail = (
    df_sales_order_detail
    .select(
        # keys
        F.col("sales_order_id").cast("int").alias("sales_order_key"),
        F.col("sales_order_detail_id").cast("int").alias("sales_order_detail_key"),
        F.col("product_id").cast("int").alias("product_key"),

        # quantities / prices
        F.col("order_qty").cast("int").alias("order_qty"),
        F.col("unit_price").cast("double").alias("unit_price"),
        F.col("unit_price_discount").cast("double").alias("unit_price_discount"),
        F.col("line_total").cast("double").alias("line_total"),

        # helpful derived measures
        (F.col("order_qty") * F.col("unit_price")).cast("double").alias("gross_line_amount"),
        (F.col("order_qty") * F.col("unit_price") * F.col("unit_price_discount")).cast("double").alias("discount_amount"),

        # audit
        F.col("modified_date")
    )
    # ensure unique grain: 1 row per sales_order_detail_id
    .dropDuplicates(["sales_order_detail_key"])
)

# --- Optional validation ---
display(fct_sales_order_detail)
fct_sales_order_detail.printSchema()

sales_order_key,sales_order_detail_key,product_key,order_qty,unit_price,unit_price_discount,line_total,gross_line_amount,discount_amount,modified_date
71782,110682,970,1,728.91,0.0,728.91,728.91,0.0,2008-06-01
71902,112971,982,2,461.694,0.0,923.388,923.388,0.0,2008-06-01
71902,113000,905,3,218.454,0.0,655.362,655.362,0.0,2008-06-01
71784,110774,979,8,445.41,0.0,3563.28,3563.28,0.0,2008-06-01
71780,110639,809,3,37.152,0.0,111.456,111.456,0.0,2008-06-01
71780,110643,869,7,41.994,0.0,293.958,293.958,0.0,2008-06-01
71782,110698,945,2,54.894,0.0,109.788,109.788,0.0,2008-06-01
71816,111457,712,4,5.394,0.0,21.576,21.576,0.0,2008-06-01
71936,113236,867,14,40.5942,0.02,556.952424,568.3188,11.366376,2008-06-01
71938,113299,997,3,323.994,0.0,971.982,971.982,0.0,2008-06-01


root
 |-- sales_order_key: integer (nullable = true)
 |-- sales_order_detail_key: integer (nullable = true)
 |-- product_key: integer (nullable = true)
 |-- order_qty: integer (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- unit_price_discount: double (nullable = true)
 |-- line_total: double (nullable = true)
 |-- gross_line_amount: double (nullable = true)
 |-- discount_amount: double (nullable = true)
 |-- modified_date: date (nullable = true)



### Define Gold output paths

This cell defines the standardised Gold-layer storage locations for all dimension and fact tables.


In [0]:
gold_root = "abfss://gold@gbosstorageaccount.dfs.core.windows.net/SalesLT"

gold_paths = {
    "dm_customer": f"{gold_root}/dm_customer/",
    "dm_address": f"{gold_root}/dm_address/",
    "dm_customer_address": f"{gold_root}/dm_customer_address/",
    "dm_product": f"{gold_root}/dm_product/",
    "dm_product_category": f"{gold_root}/dm_product_category/",
    "dm_product_model": f"{gold_root}/dm_product_model/",
    "dm_product_description": f"{gold_root}/dm_product_description/",
    "dm_product_model_product_description": f"{gold_root}/dm_product_model_product_description/",
    "fct_sales_order_header": f"{gold_root}/fct_sales_order_header/",
    "fct_sales_order_detail": f"{gold_root}/fct_sales_order_detail/",
}


### Write Gold dimension and fact tables

This cell writes all prepared Gold dimensions and fact tables to Azure Data Lake Storage as Delta tables for analytics and BI consumption.


In [0]:
gold_tables = {
    "dm_customer": dm_customer,
    "dm_address": dm_address,
    "dm_customer_address": dm_customer_address,
    "dm_product": dm_product,
    "dm_product_category": dm_product_category,
    "dm_product_model": dm_product_model,
    "dm_product_description": dm_product_description,
    "dm_product_model_product_description": dm_product_model_product_description,
    "fct_sales_order_header": fct_sales_order_header,
    "fct_sales_order_detail": fct_sales_order_detail,
}


### Validate Gold layer outputs

This cell lists the Gold container contents to confirm that all dimension and fact tables have been successfully written.


In [0]:
for table_name, df in gold_tables.items():
    (
        df.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .save(gold_paths[table_name])
    )
    print(f"✅ {table_name} written to Gold")


✅ dm_customer written to Gold
✅ dm_address written to Gold
✅ dm_customer_address written to Gold
✅ dm_product written to Gold
✅ dm_product_category written to Gold
✅ dm_product_model written to Gold
✅ dm_product_description written to Gold
✅ dm_product_model_product_description written to Gold
✅ fct_sales_order_header written to Gold
✅ fct_sales_order_detail written to Gold
